In [2]:
import re
import os
import sqlite3
from pathlib import Path
import hunparl as hp

in_dir = Path("/home/xn/Script/Streamlit/text")

con = sqlite3.connect(in_dir/"ogyn.db")
cur = con.cursor()
cur.execute("""DROP TABLE IF EXISTS ogyn""")

cur.execute("""CREATE TABLE ogyn(
            id INTEGER NOT NULL PRIMARY KEY,
            date TEXT,
            name TEXT,
            full_text TEXT)
            """)
con.commit()

In [3]:
for file in os.listdir(in_dir):
    if ".txt" in file:
        print(file)
        ogyn_id = re.findall(r'\d+', file)[0]
        with open(in_dir/file,'r') as f:
            ogyn_text = f.read()
            date = hp.ules_datum(ogyn_text)
            cur.execute(f"INSERT INTO ogyn VALUES(?,?,?,?)",
                        (ogyn_id,
                         date,
                         file,
                         ogyn_text))
            con.commit()

Országgyűlési Napló 166.szám.txt
Országgyűlési Napló 174.szám.txt
Országgyűlési Napló 165.szám.txt
Országgyűlési Napló 164.szám.txt
Országgyűlési Napló 167.szám.txt
Országgyűlési Napló 168.szám.txt
Országgyűlési Napló 177.szám.txt
Országgyűlési Napló 169.szám.txt
Országgyűlési Napló 176.szám.txt
Országgyűlési Napló 175.szám.txt
Országgyűlési Napló 172.szám.txt
Országgyűlési Napló 170.szám.txt
Országgyűlési Napló 178.szám.txt
Országgyűlési Napló 171.szám.txt
Országgyűlési Napló 162.szám.txt
Országgyűlési Napló 163.szám.txt
Országgyűlési Napló 173.szám.txt


In [11]:
f

<_io.TextIOWrapper name='/home/xn/Script/Streamlit/text/Országgyűlési Napló 166.szám.txt' mode='r' encoding='UTF-8'>

In [4]:
res = cur.execute("select * from ogyn where id =163")
text = res.fetchall()
print(text[0][1])

cleaned = hp.ogy_n_tisztazo(text[0][3])
kepviselo_lista = hp.kepviselo_lista(cleaned)
kepviseloi_felszolalas_szotar = hp.kepviseloi_felszolalas_szotar(cleaned,kepviselo_lista)
def elnok_lista(text: str)->list:
    chairman_pattern = re.compile("Napló([\s\S]*?)elnöklete alatt")
    chairman_list = re.findall(chairman_pattern, text)
    chairman_list = chairman_list[0].split("és")
    chairman_list = [i.strip().replace("\n","") for i in chairman_list]
    return chairman_list

def jegyzo_lista(text: str)->list:
    notary_pattern = re.compile("Jegyzők:([\s\S]*?)\n\n")
    notary_list = re.findall(notary_pattern, text)
    notary_list = notary_list[0].replace("  ", "")\
    .replace(" \x0c", "").split(",")
    notary_list = [i.strip().replace("\n","") for i in notary_list]
    return notary_list

2020. november 5.


In [5]:
kepviselo_lista = hp.kepviselo_lista(cleaned)
kepviseloi_felszolalas_szotar = hp.kepviseloi_felszolalas_szotar(cleaned,kepviselo_lista)

In [6]:
bev = hp.bevezeto_resz(text[0][3])

In [19]:
bev = bev.replace(".","")
page_number_pat = re.compile(r'\d{5}')
re.sub(page_number_pat, '', bev)

'   2018-2022 országgyűlési ciklus Budapest, 2020 november 5 csütörtök \n\n163 szám \n\n \n\n \n \n \n \n\n \n\nOrszággyűlési Napló \n\nDr Hiller István, Jakab István és Lezsák Sándor  \n\nelnöklete alatt \n\n \n\nJegyzők: dr Aradszki András, Tordai Bence \n\n \n\n \n \n \n \n\n \n \n \n \n \n \n \n \n \n \n \n \n \n\n \n \n \n \n \n \n \n \n\n\x0cHasáb \n\nTárgyai   \n \nAz ülésnap megnyitása   \n \nEgyes törvényeknek az otthonteremtési akcióterv bevezetése érdekében szükséges \nmódosításáról szóló törvényjavaslat általános vitája a lezárásig   \nNovák Katalin tárca nélküli miniszter, a napirendi pont előadója   \nFelszólalók: \nDunai Mónika, a Fidesz képviselőcsoportja részéről   \n \n \nZ Kárpát Dániel, a Jobbik képviselőcsoportja részéről   \n  Hollik István, a KDNP képviselőcsoportja részéről   \nDr Gurmai Zita, az MSZP képviselőcsoportja részéről   \n \nVarju László, a DK képviselőcsoportja részéről   \n \n \nCsárdi Antal, az LMP képviselőcsoportja részéről   \nZ Kárpát Dániel (J

In [14]:
cleaned_text

'   2018-2022 országgyűlési ciklus Budapest, 2020 november 5 csütörtök \n\n163 szám \n\n \n\n \n \n \n \n\n \n\nOrszággyűlési Napló \n\nDr Hiller István, Jakab István és Lezsák Sándor  \n\nelnöklete alatt \n\n \n\nJegyzők: dr Aradszki András, Tordai Bence \n\n \n\n \n \n \n \n\n \n \n \n \n \n \n \n \n \n \n \n \n \n\n \n \n \n \n \n \n \n \n\n\x0cHasáb \n\nTárgyai   \n \nAz ülésnap megnyitása  22893 \n \nEgyes törvényeknek az otthonteremtési akcióterv bevezetése érdekében szükséges \nmódosításáról szóló törvényjavaslat általános vitája a lezárásig  22893 \nNovák Katalin tárca nélküli miniszter, a napirendi pont előadója  22893 \nFelszólalók: \nDunai Mónika, a Fidesz képviselőcsoportja részéről  22897 \n \n \nZ Kárpát Dániel, a Jobbik képviselőcsoportja részéről  22901 \n  Hollik István, a KDNP képviselőcsoportja részéről  22906 \nDr Gurmai Zita, az MSZP képviselőcsoportja részéről  22911 \n \nVarju László, a DK képviselőcsoportja részéről  22913 \n \n \nCsárdi Antal, az LMP képviselőc

In [119]:
reakcio_szotar = hp.reakcio_szotar(kepviseloi_felszolalas_szotar)
reakcio_szotar

{'ARATÓ GERGELY': [['(Közbeszólás a kormánypárti padsorokból: Miért ordítasz?)',
   '(Közbeszólás a kormánypárti padsorokból.)',
   '(Németh Szilárd István: Akkor ülj  le!)',
   '(Az elnök a csengő megkocogtatásával jelzi az idő leteltét.)',
   '(Taps az ellenzéki pártok  soraiban.)'],
  ['(Kocsis Máté: Ne hazudj!)', '(Taps az ellenzéki oldalon.)']],
 'BANA TIBOR': [],
 'BENCSIK JÁNOS': [],
 'DR. FÓNAGY JÁNOS': [],
 'DR. KERESZTES LÁSZLÓ LÓRÁNT': [['(Taps az ellenzéki sorokból.)'],
  ['(Kocsis Máté közbeszól.)',
   '(Hollik István és Nacsa Lőrinc közbeszól.)',
   '(Korózs Lajos bólogat.)',
   '(Közbeszólások a kormánypárti padsorokból.)',
   '(Taps a Párbeszéd padsoraiból.)'],
  ['(Taps az ellenzéki  pártok soraiban.)'],
  ['(Taps a Jobbik padsoraiból.)']],
 'DR. LUKÁCS LÁSZLÓ GYÖRGY': [['(Taps a Jobbik padsoraiban.)'],
  ['(Kocsis Máté az ellenzék felé mutat.)', '(Taps a Jobbik padsoraiban.)']],
 'DR. MELLÁR TAMÁS': [],
 'DR. NAGY ISTVÁN': [],
 'DR. ORBÁN': [['(Németh Szilárd István: 

In [137]:
filtered_dict = {k: v for k, v in reakcio_szotar.items() if len(v) != 0}
filtered_dict.keys()

dict_keys(['ARATÓ GERGELY', 'DR. KERESZTES LÁSZLÓ LÓRÁNT', 'DR. LUKÁCS LÁSZLÓ GYÖRGY', 'DR. ORBÁN', 'DR. RÉTVÁRI BENCE', 'DR. VARGA JUDIT', 'DÖMÖTÖR CSABA', 'KOCSIS MÁTÉ', 'KORÓZS LAJOS', 'SCHANDA TAMÁS JÁNOS', 'SOLTÉSZ MIKLÓS', 'SZÉKELY SÁNDOR', 'ZSIGÓ RÓBERT'])